# 04__preprocess_mpranalyze_compare

in this notebook, i re-shape the counts data to run MPRAnalyze comparison mode. importantly, i also include the negative controls for comparison mode that I made in the previous notebook (01). 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import itertools
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import sys

from scipy.stats import spearmanr

# import utils
sys.path.append("../../../utils")
from plotting_utils import *

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.autolayout'] = False

In [2]:
sns.set(**PAPER_PRESET)
fontsize = PAPER_FONTSIZE

In [3]:
np.random.seed(2019)

## functions

In [4]:
def ctrl_status(row):
    if "CONTROL" in row.comp_id:
        return True
    else:
        return False

## variables

In [5]:
mpranalyze_dir = "../../../data/02__mpra/01__counts/mpranalyze_files"

In [6]:
dna_counts_f = "%s/dna_counts.mpranalyze.for_quantification.txt" % mpranalyze_dir
rna_counts_f = "%s/rna_counts.mpranalyze.for_quantification.txt" % mpranalyze_dir

In [7]:
data_dir = "../../../data/02__mpra/02__activs"

In [8]:
# human_max_f = "%s/human_TSS_vals.max_tile.txt" % data_dir
# mouse_max_f = "%s/mouse_TSS_vals.max_tile.txt" % data_dir

In [9]:
tss_map_f = "../../../data/01__design/01__mpra_list/mpra_tss.with_ids.UPDATED_WITH_DIV.txt"

In [10]:
dna_col_ann_f = "%s/dna_col_ann.mpranalyze.for_quantification.txt" % mpranalyze_dir
rna_col_ann_f = "%s/rna_col_ann.mpranalyze.for_quantification.txt" % mpranalyze_dir

In [11]:
human_vals_f = "%s/human_TSS_vals.both_tiles.txt" % data_dir
mouse_vals_f = "%s/mouse_TSS_vals.both_tiles.txt" % data_dir

## 1. import data

In [12]:
dna_counts = pd.read_table(dna_counts_f)
dna_counts.head()

,element,samp:dna_1__barc:1,samp:dna_1__barc:10,samp:dna_1__barc:11,samp:dna_1__barc:12,samp:dna_1__barc:13,samp:dna_1__barc:2,samp:dna_1__barc:3,samp:dna_1__barc:4,samp:dna_1__barc:5,samp:dna_1__barc:6,samp:dna_1__barc:7,samp:dna_1__barc:8,samp:dna_1__barc:9
0,AAAAAAAAAAAAAAAAACCCTGCAGAGAGCCTGCAAAGTCACTGCC...,614.0,126.0,94.0,2024.0,968.0,806.0,592.0,78.0,0.0,224.0,478.0,32.0,320.0
1,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,64.0,12.0,52.0,0.0,16.0,94.0,44.0,128.0,178.0,44.0,0.0,20.0,340.0
2,AAAAAAAAAAAAGAGGAGAAATAGATTGTTACCTTATATTATTTAA...,218.0,54.0,170.0,22.0,66.0,182.0,116.0,8.0,28.0,0.0,72.0,116.0,0.0
3,AAAAAAAAAACCGGCAAAATGTCCTTTTCCTTGTTTTGAAAAGACT...,460.0,346.0,626.0,448.0,324.0,502.0,86.0,162.0,210.0,414.0,352.0,524.0,468.0
4,AAAAAAAAAGGCCACGCTCAAAACCCCAGACTAGTTTTCCTCACCA...,596.0,242.0,456.0,180.0,0.0,828.0,226.0,42.0,302.0,446.0,750.0,540.0,650.0


In [13]:
rna_counts = pd.read_table(rna_counts_f)
rna_counts.head()

,element,samp:HUES64_rep1__barc:1,samp:HUES64_rep1__barc:10,samp:HUES64_rep1__barc:11,samp:HUES64_rep1__barc:12,samp:HUES64_rep1__barc:13,samp:HUES64_rep1__barc:2,samp:HUES64_rep1__barc:3,samp:HUES64_rep1__barc:4,samp:HUES64_rep1__barc:5,...,samp:mESC_rep3__barc:12,samp:mESC_rep3__barc:13,samp:mESC_rep3__barc:2,samp:mESC_rep3__barc:3,samp:mESC_rep3__barc:4,samp:mESC_rep3__barc:5,samp:mESC_rep3__barc:6,samp:mESC_rep3__barc:7,samp:mESC_rep3__barc:8,samp:mESC_rep3__barc:9
0,AAAAAAAAAAAAAAAAACCCTGCAGAGAGCCTGCAAAGTCACTGCC...,6863.0,2060.0,40.0,22028.0,7696.0,9308.0,4931.0,519.0,0.0,...,7989.0,2737.0,2496.0,1980.0,232.0,0.0,530.0,1062.0,74.0,1393.0
1,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,9.0,1.0,2.0,0.0,3.0,29.0,151.0,136.0,229.0,...,0.0,4.0,133.0,1.0,31.0,334.0,0.0,0.0,0.0,422.0
2,AAAAAAAAAAAAGAGGAGAAATAGATTGTTACCTTATATTATTTAA...,28.0,264.0,119.0,1.0,7.0,272.0,20.0,15.0,0.0,...,0.0,0.0,117.0,3.0,0.0,3.0,0.0,100.0,0.0,0.0
3,AAAAAAAAAACCGGCAAAATGTCCTTTTCCTTGTTTTGAAAAGACT...,175.0,16.0,789.0,755.0,661.0,415.0,50.0,7.0,600.0,...,266.0,0.0,30.0,10.0,100.0,291.0,222.0,57.0,128.0,68.0
4,AAAAAAAAAGGCCACGCTCAAAACCCCAGACTAGTTTTCCTCACCA...,1045.0,176.0,733.0,97.0,0.0,2295.0,619.0,0.0,1659.0,...,18.0,0.0,327.0,119.0,2.0,326.0,228.0,611.0,553.0,541.0


In [14]:
human_vals = pd.read_table(human_vals_f)
mouse_vals = pd.read_table(mouse_vals_f)
human_vals.head()

,HUES64,mESC,HUES64_pval,mESC_pval,HUES64_padj,mESC_padj,element,tile_type,element_id,name,...,dupe_info,HUES64_log,mESC_log,tss_id,species,tss_tile_num,hg19_id,biotype_hg19,stem_exp_hg19,orig_species
0,1.326796,1.348824,0.145102,0.232651,0.331993,0.545526,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile1:129255098-1292552...,HUMAN_EVO_TSS__h.1133__tile1,...,7281.1.0.0,0.122804,0.129955,h.1133,HUMAN,tile1,h.1133,enhancer,0.20306,human
1,1.202075,1.503351,0.242922,0.128950,0.488166,0.364500,CTACTCCGGAGGCTGAGGCAGGAGAATCGCTTGAACCCGGGGGTGG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile2:129255212-1292553...,HUMAN_EVO_TSS__h.1133__tile2,...,7282.1.0.0,0.079932,0.177061,h.1133,HUMAN,tile2,h.1133,enhancer,0.20306,human
2,1.326796,1.348824,0.145102,0.232651,0.331993,0.545526,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile1:129255098-1292552...,HUMAN_EVO_TSS__h.1133__tile1,...,7281.1.0.0,0.122804,0.129955,h.1133,HUMAN,tile1,h.1133,enhancer,0.20306,human
3,1.202075,1.503351,0.242922,0.128950,0.488166,0.364500,CTACTCCGGAGGCTGAGGCAGGAGAATCGCTTGAACCCGGGGGTGG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile2:129255212-1292553...,HUMAN_EVO_TSS__h.1133__tile2,...,7282.1.0.0,0.079932,0.177061,h.1133,HUMAN,tile2,h.1133,enhancer,0.20306,human
4,1.323999,1.457128,0.146955,0.155924,0.335623,0.417211,AAAAAAAAAGGCCACGCTCAAAACCCCAGACTAGTTTTCCTCACCA...,WILDTYPE,HUMAN_EVO_TSS__h.1436__tile1:157192113-1571922...,HUMAN_EVO_TSS__h.1436__tile1,...,4134.1.0.0,0.121888,0.163498,h.1436,HUMAN,tile1,h.1436,enhancer,0.039043800000000004,human


In [15]:
tss_map = pd.read_table(tss_map_f, index_col=0)
tss_map.head()

,start_tss_hg19,end_tss_hg19,strand_tss_hg19,cage_id_hg19,biotype_hg19,name_peak_hg19,seq_orth,cage_orth,chr_tss_mm9,start_tss_mm9,...,har,orig_species,lift_species,oligo_type,hg19_id,mm9_id,cleaner_biotype_hg19,cleaner_biotype_mm9,minimal_biotype_hg19,minimal_biotype_mm9
chr_tss_hg19,,,,,,,,,,,,,,,,,,,,,
chr1,2985430,2985431,-,"chr1:2985420..2985438,-",div_lnc,ENSG00000177133.6,True,True,chr4,154011252,...,False,human,mouse,human__divergent:mouse__divergent,h.2,m.1,div. lncRNA,div. lncRNA,lncRNA,lncRNA
chr1,8086552,8086553,+,"chr1:8086546..8086571,+",div_lnc,ENSG00000238290.1,True,True,chr4,150229044,...,False,human,mouse,human__divergent:mouse__antisense,h.3,m.2,div. lncRNA,other lncRNA,lncRNA,lncRNA
chr1,26498322,26498323,-,"chr1:26498321..26498327,-",antisense_upep,ENSG00000236782.1,True,True,chr4,133799677,...,False,human,mouse,human__antisense:mouse__protein_coding,h.4,m.3,other lncRNA,mRNA,lncRNA,mRNA
chr1,65533428,65533429,-,"chr1:65533390..65533443,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,False,human,mouse,human__intergenic:mouse__intergenic,h.5,m.4,lincRNA,other lncRNA,lncRNA,lncRNA
chr1,65533462,65533463,-,"chr1:65533457..65533465,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,False,human,mouse,human__intergenic:mouse__intergenic,h.6,m.4,lincRNA,other lncRNA,lncRNA,lncRNA


In [16]:
old_dna_col_ann = pd.read_table(dna_col_ann_f, index_col=0)
old_dna_col_ann.head()

,sample,condition,barcode
samp:dna_1__barc:1,1,dna,1
samp:dna_1__barc:10,1,dna,10
samp:dna_1__barc:11,1,dna,11
samp:dna_1__barc:12,1,dna,12
samp:dna_1__barc:13,1,dna,13


In [17]:
old_rna_col_ann = pd.read_table(rna_col_ann_f, index_col=0)
old_rna_col_ann.head()

,sample,condition,barcode
samp:HUES64_rep1__barc:1,rep1,HUES64,1
samp:HUES64_rep1__barc:10,rep1,HUES64,10
samp:HUES64_rep1__barc:11,rep1,HUES64,11
samp:HUES64_rep1__barc:12,rep1,HUES64,12
samp:HUES64_rep1__barc:13,rep1,HUES64,13


## 2. remove any sequences in TSS map that we removed at initial MPRAnalyze (low counts)

In [18]:
# filter out any elements we removed at initial steps (low dna counts)
human_vals = human_vals[human_vals["element"].isin(dna_counts["element"])]
mouse_vals = mouse_vals[mouse_vals["element"].isin(dna_counts["element"])]

## 3. get positive ctrl dna/rna counts

In [19]:
dna_counts_ctrl = dna_counts[dna_counts["element"].str.contains("samp")]
print(len(dna_counts_ctrl))
rna_counts_ctrl = rna_counts[rna_counts["element"].str.contains("samp")]
print(len(rna_counts_ctrl))

400
400


# first make files needed for seq. comparison (native and cis effects)

## 1. merge ortholog pairs w/ counts
this time, always pair tile1 with tile1 and tile2 with tile2

In [20]:
human_vals.head()

,HUES64,mESC,HUES64_pval,mESC_pval,HUES64_padj,mESC_padj,element,tile_type,element_id,name,...,dupe_info,HUES64_log,mESC_log,tss_id,species,tss_tile_num,hg19_id,biotype_hg19,stem_exp_hg19,orig_species
0,1.326796,1.348824,0.145102,0.232651,0.331993,0.545526,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile1:129255098-1292552...,HUMAN_EVO_TSS__h.1133__tile1,...,7281.1.0.0,0.122804,0.129955,h.1133,HUMAN,tile1,h.1133,enhancer,0.20306,human
1,1.202075,1.503351,0.242922,0.128950,0.488166,0.364500,CTACTCCGGAGGCTGAGGCAGGAGAATCGCTTGAACCCGGGGGTGG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile2:129255212-1292553...,HUMAN_EVO_TSS__h.1133__tile2,...,7282.1.0.0,0.079932,0.177061,h.1133,HUMAN,tile2,h.1133,enhancer,0.20306,human
2,1.326796,1.348824,0.145102,0.232651,0.331993,0.545526,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile1:129255098-1292552...,HUMAN_EVO_TSS__h.1133__tile1,...,7281.1.0.0,0.122804,0.129955,h.1133,HUMAN,tile1,h.1133,enhancer,0.20306,human
3,1.202075,1.503351,0.242922,0.128950,0.488166,0.364500,CTACTCCGGAGGCTGAGGCAGGAGAATCGCTTGAACCCGGGGGTGG...,WILDTYPE,HUMAN_EVO_TSS__h.1133__tile2:129255212-1292553...,HUMAN_EVO_TSS__h.1133__tile2,...,7282.1.0.0,0.079932,0.177061,h.1133,HUMAN,tile2,h.1133,enhancer,0.20306,human
4,1.323999,1.457128,0.146955,0.155924,0.335623,0.417211,AAAAAAAAAGGCCACGCTCAAAACCCCAGACTAGTTTTCCTCACCA...,WILDTYPE,HUMAN_EVO_TSS__h.1436__tile1:157192113-1571922...,HUMAN_EVO_TSS__h.1436__tile1,...,4134.1.0.0,0.121888,0.163498,h.1436,HUMAN,tile1,h.1436,enhancer,0.039043800000000004,human


In [21]:
dna_counts_human_all = human_vals[["element", "tss_id", "tss_tile_num"]].merge(dna_counts, on="element").drop_duplicates()
dna_counts_mouse_all = mouse_vals[["element", "tss_id", "tss_tile_num"]].merge(dna_counts, on="element").drop_duplicates()
dna_counts_human_all.head()

,element,tss_id,tss_tile_num,samp:dna_1__barc:1,samp:dna_1__barc:10,samp:dna_1__barc:11,samp:dna_1__barc:12,samp:dna_1__barc:13,samp:dna_1__barc:2,samp:dna_1__barc:3,samp:dna_1__barc:4,samp:dna_1__barc:5,samp:dna_1__barc:6,samp:dna_1__barc:7,samp:dna_1__barc:8,samp:dna_1__barc:9
0,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,h.1133,tile1,64.0,12.0,52.0,0.0,16.0,94.0,44.0,128.0,178.0,44.0,0.0,20.0,340.0
2,CTACTCCGGAGGCTGAGGCAGGAGAATCGCTTGAACCCGGGGGTGG...,h.1133,tile2,94.0,38.0,102.0,0.0,188.0,372.0,0.0,0.0,0.0,94.0,136.0,18.0,14.0
4,AAAAAAAAAGGCCACGCTCAAAACCCCAGACTAGTTTTCCTCACCA...,h.1436,tile1,596.0,242.0,456.0,180.0,0.0,828.0,226.0,42.0,302.0,446.0,750.0,540.0,650.0
6,TTAAACTCGTTTAAACCAATTTTGAATAAGCTACTTCTTAGGCTAG...,h.1436,tile2,148.0,58.0,26.0,42.0,42.0,180.0,0.0,40.0,150.0,8.0,0.0,206.0,280.0
8,AAAAAAAAGAAGAAGTGGGGCTTTCCAGAGGTGATTAAGTCATGAG...,h.1951,tile1,1540.0,3692.0,824.0,1232.0,60.0,538.0,32.0,1460.0,158.0,64.0,706.0,198.0,756.0


In [22]:
print(len(dna_counts_human_all))
print(len(dna_counts_mouse_all))

4990
4803


In [23]:
rna_counts_human_all = human_vals[["element", "tss_id", "tss_tile_num"]].merge(rna_counts, on="element").drop_duplicates()
rna_counts_mouse_all = mouse_vals[["element", "tss_id", "tss_tile_num"]].merge(rna_counts, on="element").drop_duplicates()
rna_counts_human_all.head()

,element,tss_id,tss_tile_num,samp:HUES64_rep1__barc:1,samp:HUES64_rep1__barc:10,samp:HUES64_rep1__barc:11,samp:HUES64_rep1__barc:12,samp:HUES64_rep1__barc:13,samp:HUES64_rep1__barc:2,samp:HUES64_rep1__barc:3,...,samp:mESC_rep3__barc:12,samp:mESC_rep3__barc:13,samp:mESC_rep3__barc:2,samp:mESC_rep3__barc:3,samp:mESC_rep3__barc:4,samp:mESC_rep3__barc:5,samp:mESC_rep3__barc:6,samp:mESC_rep3__barc:7,samp:mESC_rep3__barc:8,samp:mESC_rep3__barc:9
0,AAAAAAAAAAAAAAAGAAAAGAAAAGAAAAAAAAGAAAGGATTGAG...,h.1133,tile1,9.0,1.0,2.0,0.0,3.0,29.0,151.0,...,0.0,4.0,133.0,1.0,31.0,334.0,0.0,0.0,0.0,422.0
2,CTACTCCGGAGGCTGAGGCAGGAGAATCGCTTGAACCCGGGGGTGG...,h.1133,tile2,220.0,0.0,26.0,0.0,76.0,638.0,0.0,...,0.0,81.0,130.0,0.0,0.0,0.0,62.0,26.0,0.0,0.0
4,AAAAAAAAAGGCCACGCTCAAAACCCCAGACTAGTTTTCCTCACCA...,h.1436,tile1,1045.0,176.0,733.0,97.0,0.0,2295.0,619.0,...,18.0,0.0,327.0,119.0,2.0,326.0,228.0,611.0,553.0,541.0
6,TTAAACTCGTTTAAACCAATTTTGAATAAGCTACTTCTTAGGCTAG...,h.1436,tile2,27.0,0.0,69.0,223.0,121.0,76.0,0.0,...,35.0,3.0,38.0,0.0,0.0,7.0,0.0,0.0,78.0,68.0
8,AAAAAAAAGAAGAAGTGGGGCTTTCCAGAGGTGATTAAGTCATGAG...,h.1951,tile1,1995.0,2523.0,1476.0,759.0,4.0,751.0,5.0,...,383.0,0.0,398.0,1.0,1088.0,0.0,28.0,600.0,127.0,272.0


In [24]:
print(len(rna_counts_human_all))
print(len(rna_counts_mouse_all))

4990
4803


## 2. merge human/mouse counts into 1 dataframe

merge tile1 with tile1 and tile2 with tile2

In [25]:
dna_counts_human_tile1 = dna_counts_human_all[dna_counts_human_all["tss_tile_num"] == "tile1"]
dna_counts_human_tile2 = dna_counts_human_all[dna_counts_human_all["tss_tile_num"] == "tile2"]
print(len(dna_counts_human_tile1))
print(len(dna_counts_human_tile2))

2629
2361


In [26]:
rna_counts_human_tile1 = rna_counts_human_all[rna_counts_human_all["tss_tile_num"] == "tile1"]
rna_counts_human_tile2 = rna_counts_human_all[rna_counts_human_all["tss_tile_num"] == "tile2"]
print(len(rna_counts_human_tile1))
print(len(rna_counts_human_tile2))

2629
2361


In [27]:
dna_counts_mouse_tile1 = dna_counts_mouse_all[dna_counts_mouse_all["tss_tile_num"] == "tile1"]
dna_counts_mouse_tile2 = dna_counts_mouse_all[dna_counts_mouse_all["tss_tile_num"] == "tile2"]
print(len(dna_counts_mouse_tile1))
print(len(dna_counts_mouse_tile2))

2518
2285


In [28]:
rna_counts_mouse_tile1 = rna_counts_mouse_all[rna_counts_mouse_all["tss_tile_num"] == "tile1"]
rna_counts_mouse_tile2 = rna_counts_mouse_all[rna_counts_mouse_all["tss_tile_num"] == "tile2"]
print(len(rna_counts_mouse_tile1))
print(len(rna_counts_mouse_tile2))

2518
2285


In [29]:
tss_map.columns

Index(['start_tss_hg19', 'end_tss_hg19', 'strand_tss_hg19', 'cage_id_hg19',
       'biotype_hg19', 'name_peak_hg19', 'seq_orth', 'cage_orth',
       'chr_tss_mm9', 'start_tss_mm9', 'end_tss_mm9', 'strand_tss_mm9',
       'cage_id_mm9', 'biotype_mm9', 'name_peak_mm9', 'avg_exp_hg19',
       'avg_exp_mm9', 'stem_exp_hg19', 'stem_exp_mm9', 'har', 'orig_species',
       'lift_species', 'oligo_type', 'hg19_id', 'mm9_id',
       'cleaner_biotype_hg19', 'cleaner_biotype_mm9', 'minimal_biotype_hg19',
       'minimal_biotype_mm9'],
      dtype='object')

In [30]:
tss_map_mpra_tile1 = tss_map.merge(rna_counts_human_tile1, left_on="hg19_id", 
                                   right_on="tss_id").merge(rna_counts_mouse_tile1, left_on="mm9_id", right_on="tss_id",
                                                            suffixes=("___seq:human", "___seq:mouse"))
tss_map_mpra_tile1.drop_duplicates(inplace=True)
print(len(tss_map_mpra_tile1))
tss_map_mpra_tile1.head(5)

2700


,start_tss_hg19,end_tss_hg19,strand_tss_hg19,cage_id_hg19,biotype_hg19,name_peak_hg19,seq_orth,cage_orth,chr_tss_mm9,start_tss_mm9,...,samp:mESC_rep3__barc:12___seq:mouse,samp:mESC_rep3__barc:13___seq:mouse,samp:mESC_rep3__barc:2___seq:mouse,samp:mESC_rep3__barc:3___seq:mouse,samp:mESC_rep3__barc:4___seq:mouse,samp:mESC_rep3__barc:5___seq:mouse,samp:mESC_rep3__barc:6___seq:mouse,samp:mESC_rep3__barc:7___seq:mouse,samp:mESC_rep3__barc:8___seq:mouse,samp:mESC_rep3__barc:9___seq:mouse
0,2985430,2985431,-,"chr1:2985420..2985438,-",div_lnc,ENSG00000177133.6,True,True,chr4,154011252,...,733.0,1.0,349.0,875.0,40.0,22.0,146.0,118.0,284.0,694.0
1,8086552,8086553,+,"chr1:8086546..8086571,+",div_lnc,ENSG00000238290.1,True,True,chr4,150229044,...,0.0,2105.0,197.0,4166.0,157.0,978.0,286.0,268.0,0.0,0.0
2,26498322,26498323,-,"chr1:26498321..26498327,-",antisense_upep,ENSG00000236782.1,True,True,chr4,133799677,...,428.0,115.0,86.0,77.0,534.0,104.0,240.0,983.0,251.0,0.0
3,65533428,65533429,-,"chr1:65533390..65533443,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,1447.0,304.0,0.0,578.0,154.0,279.0,271.0,979.0,208.0,0.0
4,65533462,65533463,-,"chr1:65533457..65533465,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,1447.0,304.0,0.0,578.0,154.0,279.0,271.0,979.0,208.0,0.0


In [31]:
tss_map_mpra_tile2 = tss_map.merge(rna_counts_human_tile2, left_on="hg19_id", 
                                   right_on="tss_id").merge(rna_counts_mouse_tile2, left_on="mm9_id", right_on="tss_id",
                                                            suffixes=("___seq:human", "___seq:mouse"))
tss_map_mpra_tile2.drop_duplicates(inplace=True)
print(len(tss_map_mpra_tile2))
tss_map_mpra_tile2.head(5)

2244


,start_tss_hg19,end_tss_hg19,strand_tss_hg19,cage_id_hg19,biotype_hg19,name_peak_hg19,seq_orth,cage_orth,chr_tss_mm9,start_tss_mm9,...,samp:mESC_rep3__barc:12___seq:mouse,samp:mESC_rep3__barc:13___seq:mouse,samp:mESC_rep3__barc:2___seq:mouse,samp:mESC_rep3__barc:3___seq:mouse,samp:mESC_rep3__barc:4___seq:mouse,samp:mESC_rep3__barc:5___seq:mouse,samp:mESC_rep3__barc:6___seq:mouse,samp:mESC_rep3__barc:7___seq:mouse,samp:mESC_rep3__barc:8___seq:mouse,samp:mESC_rep3__barc:9___seq:mouse
0,2985430,2985431,-,"chr1:2985420..2985438,-",div_lnc,ENSG00000177133.6,True,True,chr4,154011252,...,41.0,155.0,117.0,9.0,116.0,57.0,200.0,698.0,179.0,0.0
1,26498322,26498323,-,"chr1:26498321..26498327,-",antisense_upep,ENSG00000236782.1,True,True,chr4,133799677,...,142.0,950.0,49.0,298.0,779.0,246.0,745.0,65.0,393.0,892.0
2,65533428,65533429,-,"chr1:65533390..65533443,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,2723.0,717.0,424.0,6732.0,0.0,132.0,2619.0,2712.0,4485.0,6451.0
3,65533462,65533463,-,"chr1:65533457..65533465,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,2723.0,717.0,424.0,6732.0,0.0,132.0,2619.0,2712.0,4485.0,6451.0
4,95975698,95975699,+,"chr1:95975684..95975731,+",intergenic_upep,ENSG00000228971.2,True,True,chr3,120589658,...,76.0,0.0,4.0,214.0,249.0,108.0,147.0,555.0,0.0,352.0


In [32]:
tss_map_dna_tile1 = tss_map.merge(dna_counts_human_tile1, left_on="hg19_id", 
                                  right_on="tss_id").merge(dna_counts_mouse_tile1, left_on="mm9_id", right_on="tss_id",
                                                           suffixes=("___seq:human", "___seq:mouse"))
tss_map_dna_tile1.drop_duplicates(inplace=True)
print(len(tss_map_dna_tile1))
tss_map_dna_tile1.head(5)

2700


,start_tss_hg19,end_tss_hg19,strand_tss_hg19,cage_id_hg19,biotype_hg19,name_peak_hg19,seq_orth,cage_orth,chr_tss_mm9,start_tss_mm9,...,samp:dna_1__barc:12___seq:mouse,samp:dna_1__barc:13___seq:mouse,samp:dna_1__barc:2___seq:mouse,samp:dna_1__barc:3___seq:mouse,samp:dna_1__barc:4___seq:mouse,samp:dna_1__barc:5___seq:mouse,samp:dna_1__barc:6___seq:mouse,samp:dna_1__barc:7___seq:mouse,samp:dna_1__barc:8___seq:mouse,samp:dna_1__barc:9___seq:mouse
0,2985430,2985431,-,"chr1:2985420..2985438,-",div_lnc,ENSG00000177133.6,True,True,chr4,154011252,...,2336.0,126.0,1098.0,1518.0,226.0,176.0,150.0,372.0,550.0,2208.0
1,8086552,8086553,+,"chr1:8086546..8086571,+",div_lnc,ENSG00000238290.1,True,True,chr4,150229044,...,0.0,1212.0,200.0,2244.0,138.0,590.0,422.0,206.0,74.0,38.0
2,26498322,26498323,-,"chr1:26498321..26498327,-",antisense_upep,ENSG00000236782.1,True,True,chr4,133799677,...,442.0,296.0,288.0,330.0,1914.0,228.0,446.0,1158.0,358.0,190.0
3,65533428,65533429,-,"chr1:65533390..65533443,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,1060.0,256.0,0.0,250.0,316.0,408.0,352.0,1152.0,310.0,54.0
4,65533462,65533463,-,"chr1:65533457..65533465,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,1060.0,256.0,0.0,250.0,316.0,408.0,352.0,1152.0,310.0,54.0


In [33]:
tss_map_dna_tile2 = tss_map.merge(dna_counts_human_tile2, left_on="hg19_id", 
                                  right_on="tss_id").merge(dna_counts_mouse_tile2, left_on="mm9_id", right_on="tss_id",
                                                           suffixes=("___seq:human", "___seq:mouse"))
tss_map_dna_tile2.drop_duplicates(inplace=True)
print(len(tss_map_dna_tile2))
tss_map_dna_tile2.head(5)

2244


,start_tss_hg19,end_tss_hg19,strand_tss_hg19,cage_id_hg19,biotype_hg19,name_peak_hg19,seq_orth,cage_orth,chr_tss_mm9,start_tss_mm9,...,samp:dna_1__barc:12___seq:mouse,samp:dna_1__barc:13___seq:mouse,samp:dna_1__barc:2___seq:mouse,samp:dna_1__barc:3___seq:mouse,samp:dna_1__barc:4___seq:mouse,samp:dna_1__barc:5___seq:mouse,samp:dna_1__barc:6___seq:mouse,samp:dna_1__barc:7___seq:mouse,samp:dna_1__barc:8___seq:mouse,samp:dna_1__barc:9___seq:mouse
0,2985430,2985431,-,"chr1:2985420..2985438,-",div_lnc,ENSG00000177133.6,True,True,chr4,154011252,...,230.0,544.0,234.0,164.0,244.0,390.0,848.0,1376.0,624.0,60.0
1,26498322,26498323,-,"chr1:26498321..26498327,-",antisense_upep,ENSG00000236782.1,True,True,chr4,133799677,...,90.0,1856.0,226.0,362.0,2012.0,414.0,1182.0,154.0,1558.0,1428.0
2,65533428,65533429,-,"chr1:65533390..65533443,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,1096.0,296.0,166.0,1798.0,0.0,54.0,780.0,1336.0,1494.0,2028.0
3,65533462,65533463,-,"chr1:65533457..65533465,-",intergenic,ENSG00000231485.1,True,True,chr4,101029317,...,1096.0,296.0,166.0,1798.0,0.0,54.0,780.0,1336.0,1494.0,2028.0
4,95975698,95975699,+,"chr1:95975684..95975731,+",intergenic_upep,ENSG00000228971.2,True,True,chr3,120589658,...,182.0,354.0,22.0,1106.0,36.0,400.0,750.0,1678.0,42.0,336.0


## 3. assign each pair an ID

In [34]:
HUES64_rna_cols = [x for x in tss_map_mpra_tile1.columns if "samp:HUES64" in x]
mESC_rna_cols = [x for x in tss_map_mpra_tile1.columns if "samp:mESC" in x]
all_dna_cols = [x for x in tss_map_dna_tile1.columns if "samp:dna" in x]

human_cols = ["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9"]
human_cols.extend(HUES64_rna_cols)

mouse_cols = ["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9"]
mouse_cols.extend(mESC_rna_cols)

dna_cols = ["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9"]
dna_cols.extend(all_dna_cols)

tss_map_mpra_human_tile1 = tss_map_mpra_tile1[human_cols]
tss_map_mpra_mouse_tile1 = tss_map_mpra_tile1[mouse_cols]

tss_map_mpra_human_tile2 = tss_map_mpra_tile2[human_cols]
tss_map_mpra_mouse_tile2 = tss_map_mpra_tile2[mouse_cols]

tss_map_dna_tile1 = tss_map_dna_tile1[dna_cols]
tss_map_dna_tile2 = tss_map_dna_tile2[dna_cols]

tss_map_mpra_human_tile1.head()

,hg19_id,biotype_hg19,mm9_id,biotype_mm9,samp:HUES64_rep1__barc:1___seq:human,samp:HUES64_rep1__barc:10___seq:human,samp:HUES64_rep1__barc:11___seq:human,samp:HUES64_rep1__barc:12___seq:human,samp:HUES64_rep1__barc:13___seq:human,samp:HUES64_rep1__barc:2___seq:human,...,samp:HUES64_rep3__barc:12___seq:mouse,samp:HUES64_rep3__barc:13___seq:mouse,samp:HUES64_rep3__barc:2___seq:mouse,samp:HUES64_rep3__barc:3___seq:mouse,samp:HUES64_rep3__barc:4___seq:mouse,samp:HUES64_rep3__barc:5___seq:mouse,samp:HUES64_rep3__barc:6___seq:mouse,samp:HUES64_rep3__barc:7___seq:mouse,samp:HUES64_rep3__barc:8___seq:mouse,samp:HUES64_rep3__barc:9___seq:mouse
0,h.2,div_lnc,m.1,div_lnc,8374.0,3343.0,6548.0,436.0,2117.0,2659.0,...,4038.0,207.0,1929.0,2188.0,199.0,146.0,116.0,612.0,722.0,2938.0
1,h.3,div_lnc,m.2,antisense,1.0,0.0,18.0,0.0,149.0,1.0,...,0.0,1973.0,681.0,4594.0,250.0,1068.0,543.0,462.0,0.0,43.0
2,h.4,antisense_upep,m.3,protein_coding,1.0,1029.0,141.0,1729.0,2549.0,995.0,...,263.0,406.0,302.0,354.0,2587.0,502.0,351.0,2488.0,840.0,0.0
3,h.5,intergenic,m.4,intergenic_upep,3487.0,433.0,3121.0,351.0,11277.0,1028.0,...,4111.0,811.0,0.0,1075.0,1158.0,1241.0,929.0,4320.0,775.0,77.0
4,h.6,intergenic,m.4,intergenic_upep,14595.0,25119.0,9055.0,6041.0,10936.0,13086.0,...,4111.0,811.0,0.0,1075.0,1158.0,1241.0,929.0,4320.0,775.0,77.0


In [35]:
tss_map_mpra_human_tile1["tile_num"] = "tile1"
tss_map_mpra_mouse_tile1["tile_num"] = "tile1"
tss_map_mpra_human_tile2["tile_num"] = "tile2"
tss_map_mpra_mouse_tile2["tile_num"] = "tile2"
tss_map_dna_tile1["tile_num"] = "tile1"
tss_map_dna_tile2["tile_num"] = "tile2"

In [36]:
# all tile 1s
tss_map_mpra_human_tile1["comp_id"] = tss_map_mpra_human_tile1["hg19_id"] + "__" + tss_map_mpra_human_tile1["biotype_hg19"] + "__" + tss_map_mpra_human_tile1["mm9_id"] + "__" + tss_map_mpra_human_tile1["biotype_mm9"] + "__" + tss_map_mpra_human_tile1["tile_num"] 
tss_map_mpra_mouse_tile1["comp_id"] = tss_map_mpra_mouse_tile1["hg19_id"] + "__" + tss_map_mpra_mouse_tile1["biotype_hg19"] + "__" + tss_map_mpra_mouse_tile1["mm9_id"] + "__" + tss_map_mpra_mouse_tile1["biotype_mm9"] + "__" + tss_map_mpra_mouse_tile1["tile_num"]
tss_map_dna_tile1["comp_id"] = tss_map_dna_tile1["hg19_id"] + "__" + tss_map_dna_tile1["biotype_hg19"] + "__" + tss_map_dna_tile1["mm9_id"] + "__" + tss_map_dna_tile1["biotype_mm9"] + "__" + tss_map_dna_tile1["tile_num"]

# all tile 2s
tss_map_mpra_human_tile2["comp_id"] = tss_map_mpra_human_tile2["hg19_id"] + "__" + tss_map_mpra_human_tile2["biotype_hg19"] + "__" + tss_map_mpra_human_tile2["mm9_id"] + "__" + tss_map_mpra_human_tile2["biotype_mm9"] + "__" + tss_map_mpra_human_tile2["tile_num"] 
tss_map_mpra_mouse_tile2["comp_id"] = tss_map_mpra_mouse_tile2["hg19_id"] + "__" + tss_map_mpra_mouse_tile2["biotype_hg19"] + "__" + tss_map_mpra_mouse_tile2["mm9_id"] + "__" + tss_map_mpra_mouse_tile2["biotype_mm9"] + "__" + tss_map_mpra_mouse_tile2["tile_num"]
tss_map_dna_tile2["comp_id"] = tss_map_dna_tile2["hg19_id"] + "__" + tss_map_dna_tile2["biotype_hg19"] + "__" + tss_map_dna_tile2["mm9_id"] + "__" + tss_map_dna_tile2["biotype_mm9"] + "__" + tss_map_dna_tile2["tile_num"]

# drop redundant tiles
tss_map_mpra_human_tile1.drop(["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9", "tile_num"], axis=1, inplace=True)
tss_map_mpra_mouse_tile1.drop(["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9", "tile_num"], axis=1, inplace=True)
tss_map_dna_tile1.drop(["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9", "tile_num"], axis=1, inplace=True)
tss_map_mpra_human_tile2.drop(["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9", "tile_num"], axis=1, inplace=True)
tss_map_mpra_mouse_tile2.drop(["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9", "tile_num"], axis=1, inplace=True)
tss_map_dna_tile2.drop(["hg19_id", "biotype_hg19", "mm9_id", "biotype_mm9", "tile_num"], axis=1, inplace=True)

human_cols = ["comp_id"]
human_cols.extend(HUES64_rna_cols)

mouse_cols = ["comp_id"]
mouse_cols.extend(mESC_rna_cols)

dna_cols = ["comp_id"]
dna_cols.extend(all_dna_cols)

tss_map_mpra_human_tile1 = tss_map_mpra_human_tile1[human_cols]
tss_map_mpra_human_tile2 = tss_map_mpra_human_tile2[human_cols]
tss_map_mpra_mouse_tile1 = tss_map_mpra_mouse_tile1[mouse_cols]
tss_map_mpra_mouse_tile2 = tss_map_mpra_mouse_tile2[mouse_cols]
tss_map_dna_tile1 = tss_map_dna_tile1[dna_cols]
tss_map_dna_tile2 = tss_map_dna_tile2[dna_cols]

tss_map_mpra_human_tile1.head()

,comp_id,samp:HUES64_rep1__barc:1___seq:human,samp:HUES64_rep1__barc:10___seq:human,samp:HUES64_rep1__barc:11___seq:human,samp:HUES64_rep1__barc:12___seq:human,samp:HUES64_rep1__barc:13___seq:human,samp:HUES64_rep1__barc:2___seq:human,samp:HUES64_rep1__barc:3___seq:human,samp:HUES64_rep1__barc:4___seq:human,samp:HUES64_rep1__barc:5___seq:human,...,samp:HUES64_rep3__barc:12___seq:mouse,samp:HUES64_rep3__barc:13___seq:mouse,samp:HUES64_rep3__barc:2___seq:mouse,samp:HUES64_rep3__barc:3___seq:mouse,samp:HUES64_rep3__barc:4___seq:mouse,samp:HUES64_rep3__barc:5___seq:mouse,samp:HUES64_rep3__barc:6___seq:mouse,samp:HUES64_rep3__barc:7___seq:mouse,samp:HUES64_rep3__barc:8___seq:mouse,samp:HUES64_rep3__barc:9___seq:mouse
0,h.2__div_lnc__m.1__div_lnc__tile1,8374.0,3343.0,6548.0,436.0,2117.0,2659.0,0.0,48.0,347.0,...,4038.0,207.0,1929.0,2188.0,199.0,146.0,116.0,612.0,722.0,2938.0
1,h.3__div_lnc__m.2__antisense__tile1,1.0,0.0,18.0,0.0,149.0,1.0,5.0,0.0,278.0,...,0.0,1973.0,681.0,4594.0,250.0,1068.0,543.0,462.0,0.0,43.0
2,h.4__antisense_upep__m.3__protein_coding__tile1,1.0,1029.0,141.0,1729.0,2549.0,995.0,0.0,139.0,71.0,...,263.0,406.0,302.0,354.0,2587.0,502.0,351.0,2488.0,840.0,0.0
3,h.5__intergenic__m.4__intergenic_upep__tile1,3487.0,433.0,3121.0,351.0,11277.0,1028.0,10653.0,2691.0,1426.0,...,4111.0,811.0,0.0,1075.0,1158.0,1241.0,929.0,4320.0,775.0,77.0
4,h.6__intergenic__m.4__intergenic_upep__tile1,14595.0,25119.0,9055.0,6041.0,10936.0,13086.0,9383.0,3051.0,1248.0,...,4111.0,811.0,0.0,1075.0,1158.0,1241.0,929.0,4320.0,775.0,77.0


In [37]:
# append tile 1 and tile2
tss_map_mpra_human = tss_map_mpra_human_tile1.append(tss_map_mpra_human_tile2).drop_duplicates()
tss_map_mpra_mouse = tss_map_mpra_mouse_tile1.append(tss_map_mpra_mouse_tile2).drop_duplicates()
tss_map_dna = tss_map_dna_tile1.append(tss_map_dna_tile2).drop_duplicates()
print(len(tss_map_mpra_human))
print(len(tss_map_mpra_mouse))
print(len(tss_map_dna))
tss_map_mpra_human.sample(5)

4916
4916
4916


,comp_id,samp:HUES64_rep1__barc:1___seq:human,samp:HUES64_rep1__barc:10___seq:human,samp:HUES64_rep1__barc:11___seq:human,samp:HUES64_rep1__barc:12___seq:human,samp:HUES64_rep1__barc:13___seq:human,samp:HUES64_rep1__barc:2___seq:human,samp:HUES64_rep1__barc:3___seq:human,samp:HUES64_rep1__barc:4___seq:human,samp:HUES64_rep1__barc:5___seq:human,...,samp:HUES64_rep3__barc:12___seq:mouse,samp:HUES64_rep3__barc:13___seq:mouse,samp:HUES64_rep3__barc:2___seq:mouse,samp:HUES64_rep3__barc:3___seq:mouse,samp:HUES64_rep3__barc:4___seq:mouse,samp:HUES64_rep3__barc:5___seq:mouse,samp:HUES64_rep3__barc:6___seq:mouse,samp:HUES64_rep3__barc:7___seq:mouse,samp:HUES64_rep3__barc:8___seq:mouse,samp:HUES64_rep3__barc:9___seq:mouse
2253,h.2528__enhancer__m.2340__no cage activity__tile1,509.0,266.0,172.0,323.0,8.0,529.0,0.0,3.0,167.0,...,60.0,353.0,1304.0,20.0,584.0,0.0,74.0,401.0,165.0,47.0
258,h.425__intergenic__m.289__enhancer__tile2,8.0,1121.0,159.0,161.0,1261.0,581.0,450.0,786.0,774.0,...,1253.0,202.0,935.0,229.0,299.0,140.0,161.0,66.0,105.0,227.0
2487,h.3211__no cage activity__m.3022__intergenic__...,1071.0,601.0,1546.0,1138.0,1181.0,1158.0,751.0,618.0,2151.0,...,3345.0,983.0,1396.0,665.0,2234.0,1315.0,4592.0,2670.0,2537.0,2102.0
2182,h.2681__no cage activity__m.2493__antisense_up...,87.0,41.0,2320.0,1.0,0.0,342.0,1683.0,1.0,0.0,...,302.0,4.0,67.0,37.0,362.0,698.0,313.0,1647.0,1191.0,0.0
1420,h.1313__enhancer__m.1185__enhancer__tile1,2.0,1509.0,1281.0,0.0,2300.0,724.0,1637.0,642.0,0.0,...,392.0,337.0,1491.0,0.0,61.0,515.0,529.0,0.0,817.0,38.0


In [38]:
# merge human and mouse so both cols in 1 df
tss_map_mpra = tss_map_mpra_human.merge(tss_map_mpra_mouse, on="comp_id")
len(tss_map_mpra)

4916

In [39]:
# also add dataframe for native comparisons
native_cols = ["comp_id"]
native_human_cols = [x for x in tss_map_mpra.columns if "HUES64" in x and "human" in x]
native_mouse_cols = [x for x in tss_map_mpra.columns if "mESC" in x and "mouse" in x]
native_cols.extend(native_human_cols)
native_cols.extend(native_mouse_cols)
tss_map_mpra_native = tss_map_mpra[native_cols]
tss_map_mpra_native.head()

,comp_id,samp:HUES64_rep1__barc:1___seq:human,samp:HUES64_rep1__barc:10___seq:human,samp:HUES64_rep1__barc:11___seq:human,samp:HUES64_rep1__barc:12___seq:human,samp:HUES64_rep1__barc:13___seq:human,samp:HUES64_rep1__barc:2___seq:human,samp:HUES64_rep1__barc:3___seq:human,samp:HUES64_rep1__barc:4___seq:human,samp:HUES64_rep1__barc:5___seq:human,...,samp:mESC_rep3__barc:12___seq:mouse,samp:mESC_rep3__barc:13___seq:mouse,samp:mESC_rep3__barc:2___seq:mouse,samp:mESC_rep3__barc:3___seq:mouse,samp:mESC_rep3__barc:4___seq:mouse,samp:mESC_rep3__barc:5___seq:mouse,samp:mESC_rep3__barc:6___seq:mouse,samp:mESC_rep3__barc:7___seq:mouse,samp:mESC_rep3__barc:8___seq:mouse,samp:mESC_rep3__barc:9___seq:mouse
0,h.2__div_lnc__m.1__div_lnc__tile1,8374.0,3343.0,6548.0,436.0,2117.0,2659.0,0.0,48.0,347.0,...,733.0,1.0,349.0,875.0,40.0,22.0,146.0,118.0,284.0,694.0
1,h.3__div_lnc__m.2__antisense__tile1,1.0,0.0,18.0,0.0,149.0,1.0,5.0,0.0,278.0,...,0.0,2105.0,197.0,4166.0,157.0,978.0,286.0,268.0,0.0,0.0
2,h.4__antisense_upep__m.3__protein_coding__tile1,1.0,1029.0,141.0,1729.0,2549.0,995.0,0.0,139.0,71.0,...,428.0,115.0,86.0,77.0,534.0,104.0,240.0,983.0,251.0,0.0
3,h.5__intergenic__m.4__intergenic_upep__tile1,3487.0,433.0,3121.0,351.0,11277.0,1028.0,10653.0,2691.0,1426.0,...,1447.0,304.0,0.0,578.0,154.0,279.0,271.0,979.0,208.0,0.0
4,h.6__intergenic__m.4__intergenic_upep__tile1,14595.0,25119.0,9055.0,6041.0,10936.0,13086.0,9383.0,3051.0,1248.0,...,1447.0,304.0,0.0,578.0,154.0,279.0,271.0,979.0,208.0,0.0


In [40]:
# remove duplicates
tss_map_dna.drop_duplicates(inplace=True)
print(len(tss_map_dna))
print(len(tss_map_dna["comp_id"].unique()))

tss_map_mpra_human.drop_duplicates(inplace=True)
print(len(tss_map_mpra_human))
print(len(tss_map_mpra_human["comp_id"].unique()))

tss_map_mpra_mouse.drop_duplicates(inplace=True)
print(len(tss_map_mpra_mouse))
print(len(tss_map_mpra_mouse["comp_id"].unique()))

tss_map_mpra_native.drop_duplicates(inplace=True)
print(len(tss_map_mpra_native))
print(len(tss_map_mpra_native["comp_id"].unique()))

4916
4916
4916
4916
4916
4916
4916
4916


## 4. pair positive controls together to serve as negative controls
for each down-sampled control element (there are 4), randomly choose 100 pairs to serve as human/mouse

In [41]:
ctrl_ids = rna_counts_ctrl.element.unique()
ctrl_ids[0:5]

array([ 'AGTTCCGCTTACATAACTTACGGTAAATGGCCCGCCTGGCTGACCGCCCAACGACCCCCGCCCATTGACGTCAATAATGACGTATGTTCCCATAGTAACGCCAATAGGGACTTTCCATTGACGTCAATGGGTGGAGTATTTACG__samp1',
       'AGTTCCGCTTACATAACTTACGGTAAATGGCCCGCCTGGCTGACCGCCCAACGACCCCCGCCCATTGACGTCAATAATGACGTATGTTCCCATAGTAACGCCAATAGGGACTTTCCATTGACGTCAATGGGTGGAGTATTTACG__samp2',
       'AGTTCCGCTTACATAACTTACGGTAAATGGCCCGCCTGGCTGACCGCCCAACGACCCCCGCCCATTGACGTCAATAATGACGTATGTTCCCATAGTAACGCCAATAGGGACTTTCCATTGACGTCAATGGGTGGAGTATTTACG__samp3',
       'AGTTCCGCTTACATAACTTACGGTAAATGGCCCGCCTGGCTGACCGCCCAACGACCCCCGCCCATTGACGTCAATAATGACGTATGTTCCCATAGTAACGCCAATAGGGACTTTCCATTGACGTCAATGGGTGGAGTATTTACG__samp4',
       'AGTTCCGCTTACATAACTTACGGTAAATGGCCCGCCTGGCTGACCGCCCAACGACCCCCGCCCATTGACGTCAATAATGACGTATGTTCCCATAGTAACGCCAATAGGGACTTTCCATTGACGTCAATGGGTGGAGTATTTACG__samp5'], dtype=object)

In [42]:
ctrl_seqs = set([x.split("__")[0] for x in ctrl_ids])
samp_ids = set([x.split("__")[1] for x in ctrl_ids])

In [43]:
all_samp_id_pairs = list(itertools.combinations(samp_ids, 2))
all_samp_id_pairs_str = ["%s__%s" % (x[0], x[1]) for x in all_samp_id_pairs]
all_samp_id_pairs_str[0:5]

['samp47__samp25',
 'samp47__samp51',
 'samp47__samp90',
 'samp47__samp9',
 'samp47__samp28']

In [44]:
sampled_samp_id_pairs = np.random.choice(all_samp_id_pairs_str, size=100)
sampled_samp_id_pairs[0:5]

array(['samp54__samp26', 'samp66__samp62', 'samp93__samp85',
       'samp2__samp56', 'samp30__samp21'],
      dtype='<U15')

In [45]:
neg_ctrls_dna = pd.DataFrame()
neg_ctrls_human = pd.DataFrame()
neg_ctrls_mouse = pd.DataFrame()
neg_ctrls_native = pd.DataFrame()

for i, seq in enumerate(ctrl_seqs):
    print("ctrl #: %s" % (i+1))
    
    for j, samp_id_pair in enumerate(sampled_samp_id_pairs):
        if j % 50 == 0:
            print("...samp pair #: %s" % (j+1))
            
        samp1 = samp_id_pair.split("__")[0] # arbitrarily call 'human' seq
        samp2 = samp_id_pair.split("__")[1] # arbitrarily call 'mouse' seq
        
        human_elem = "%s__%s" % (seq, samp1)
        mouse_elem = "%s__%s" % (seq, samp2)
        
        human_sub_dna = dna_counts_ctrl[dna_counts_ctrl["element"] == human_elem]
        mouse_sub_dna = dna_counts_ctrl[dna_counts_ctrl["element"] == mouse_elem]
        
        human_sub_rna = rna_counts_ctrl[rna_counts_ctrl["element"] == human_elem]
        mouse_sub_rna = rna_counts_ctrl[rna_counts_ctrl["element"] == mouse_elem]
        
        # re-name columns w/ species name
        human_dna_cols = ["element"]
        mouse_dna_cols = ["element"]
        human_rna_cols = ["element"]
        mouse_rna_cols = ["element"]
        
        human_dna_cols.extend(["%s___seq:human" % x for x in human_sub_dna.columns if x != "element"])
        mouse_dna_cols.extend(["%s___seq:mouse" % x for x in mouse_sub_dna.columns if x != "element"])
        
        human_rna_cols.extend(["%s___seq:human" % x for x in human_sub_rna.columns if x != "element"])
        mouse_rna_cols.extend(["%s___seq:mouse" % x for x in mouse_sub_rna.columns if x != "element"])
        
        human_sub_dna.columns = human_dna_cols
        mouse_sub_dna.columns = mouse_dna_cols
        human_sub_rna.columns = human_rna_cols
        mouse_sub_rna.columns = mouse_rna_cols
        
        # add comp_id to each df
        comp_id = "CONTROL:%s__SAMP_PAIR:%s" % ((i+1), (j+1))
        human_sub_dna["comp_id"] = comp_id
        mouse_sub_dna["comp_id"] = comp_id
        human_sub_rna["comp_id"] = comp_id
        mouse_sub_rna["comp_id"] = comp_id
        
        # merge each df into 1
        human_sub_dna.drop("element", axis=1, inplace=True)
        mouse_sub_dna.drop("element", axis=1, inplace=True)
        human_sub_rna.drop("element", axis=1, inplace=True)
        mouse_sub_rna.drop("element", axis=1, inplace=True)
        
        sub_dna = human_sub_dna.merge(mouse_sub_dna, on="comp_id")
        sub_rna = human_sub_rna.merge(mouse_sub_rna, on="comp_id")
        
        # subset rna appropriately into each negative control bucket
        sub_rna_human_cols = [x for x in sub_rna.columns if x == "comp_id" or "HUES64" in x]
        sub_rna_mouse_cols = [x for x in sub_rna.columns if x == "comp_id" or "mESC" in x]
        sub_rna_native_cols = [x for x in sub_rna.columns if x == "comp_id" or ("HUES64" in x and "human" in x) or ("mESC" in x and "mouse" in x)]
        
        sub_rna_human = sub_rna[sub_rna_human_cols]
        sub_rna_mouse = sub_rna[sub_rna_mouse_cols]
        sub_rna_native = sub_rna[sub_rna_native_cols]
        
        # append
        neg_ctrls_dna = neg_ctrls_dna.append(sub_dna)
        neg_ctrls_human = neg_ctrls_human.append(sub_rna_human)
        neg_ctrls_mouse = neg_ctrls_mouse.append(sub_rna_mouse)
        neg_ctrls_native = neg_ctrls_native.append(sub_rna_native)

ctrl #: 1
...samp pair #: 1
...samp pair #: 51
ctrl #: 2
...samp pair #: 1
...samp pair #: 51
ctrl #: 3
...samp pair #: 1
...samp pair #: 51
ctrl #: 4
...samp pair #: 1
...samp pair #: 51


In [46]:
all_dna = tss_map_dna.append(neg_ctrls_dna)
all_dna.set_index("comp_id", inplace=True)
len(all_dna)

5316

In [47]:
all_rna_human = tss_map_mpra_human.append(neg_ctrls_human)
all_rna_human.set_index("comp_id", inplace=True)
len(all_rna_human)

5316

In [48]:
all_rna_mouse = tss_map_mpra_mouse.append(neg_ctrls_mouse)
all_rna_mouse.set_index("comp_id", inplace=True)
len(all_rna_mouse)

5316

In [49]:
all_rna_native = tss_map_mpra_native.append(neg_ctrls_native)
all_rna_native.set_index("comp_id", inplace=True)
len(all_rna_native)

5316

In [50]:
# also make file w/ everything together to test interactions!
tmp_human = all_rna_human.reset_index()
tmp_mouse = all_rna_mouse.reset_index()
all_rna = tmp_human.merge(tmp_mouse, on="comp_id")
all_cols = all_rna.columns
all_rna.set_index("comp_id", inplace=True)
len(all_rna)

5316

## 5. make annotation files

In [51]:
dna_col_ann = {}
human_col_ann = {}
mouse_col_ann = {}
native_col_ann = {}
all_col_ann = {}

for cols, ann in zip([all_dna_cols, human_cols, mouse_cols, native_cols, all_cols], 
                     [dna_col_ann, human_col_ann, mouse_col_ann, native_col_ann, all_col_ann]):
    for col in cols:
        if col == "comp_id":
            continue
        cond = col.split(":")[1].split("_")[0]
        barc = col.split(":")[2].split("_")[0]
        seq = col.split(":")[-1]
        ann[col] = {"condition": cond, "barcode": barc, "seq": seq}

dna_col_ann = pd.DataFrame.from_dict(dna_col_ann, orient="index")
human_col_ann = pd.DataFrame.from_dict(human_col_ann, orient="index")
mouse_col_ann = pd.DataFrame.from_dict(mouse_col_ann, orient="index")
native_col_ann = pd.DataFrame.from_dict(native_col_ann, orient="index")
all_col_ann = pd.DataFrame.from_dict(all_col_ann, orient="index")
native_col_ann.sample(5)

,condition,barcode,seq
samp:HUES64_rep3__barc:12___seq:human,HUES64,12,human
samp:mESC_rep1__barc:5___seq:mouse,mESC,5,mouse
samp:HUES64_rep1__barc:3___seq:human,HUES64,3,human
samp:HUES64_rep3__barc:7___seq:human,HUES64,7,human
samp:HUES64_rep3__barc:4___seq:human,HUES64,4,human


In [52]:
# merge w/ older annotations: first reset index
human_col_ann.reset_index(inplace=True)
mouse_col_ann.reset_index(inplace=True)
native_col_ann.reset_index(inplace=True)
all_col_ann.reset_index(inplace=True)

human_col_ann["colname"] = human_col_ann["index"]
mouse_col_ann["colname"] = mouse_col_ann["index"]
native_col_ann["colname"] = native_col_ann["index"]
all_col_ann["colname"] = all_col_ann["index"]

In [53]:
# reset index on old annots and turn barcode into str
old_rna_col_ann.reset_index(inplace=True)
old_rna_col_ann["barcode"] = old_rna_col_ann["barcode"].astype(str)

In [54]:
# merge
human_col_ann.sample(5)

,index,condition,barcode,seq,colname
27,samp:HUES64_rep2__barc:10___seq:mouse,HUES64,10,mouse,samp:HUES64_rep2__barc:10___seq:mouse
40,samp:HUES64_rep2__barc:4___seq:human,HUES64,4,human,samp:HUES64_rep2__barc:4___seq:human
49,samp:HUES64_rep2__barc:8___seq:mouse,HUES64,8,mouse,samp:HUES64_rep2__barc:8___seq:mouse
6,samp:HUES64_rep1__barc:13___seq:human,HUES64,13,human,samp:HUES64_rep1__barc:13___seq:human
59,samp:HUES64_rep3__barc:13___seq:mouse,HUES64,13,mouse,samp:HUES64_rep3__barc:13___seq:mouse


In [55]:
all_col_ann.sample(5)

,index,condition,barcode,seq,colname
142,samp:mESC_rep3__barc:3___seq:human,mESC,3,human,samp:mESC_rep3__barc:3___seq:human
150,samp:mESC_rep3__barc:7___seq:human,mESC,7,human,samp:mESC_rep3__barc:7___seq:human
88,samp:mESC_rep1__barc:2___seq:human,mESC,2,human,samp:mESC_rep1__barc:2___seq:human
100,samp:mESC_rep1__barc:8___seq:human,mESC,8,human,samp:mESC_rep1__barc:8___seq:human
116,samp:mESC_rep2__barc:3___seq:human,mESC,3,human,samp:mESC_rep2__barc:3___seq:human


In [56]:
# reset index
human_col_ann.set_index("colname", inplace=True)
mouse_col_ann.set_index("colname", inplace=True)
native_col_ann.set_index("colname", inplace=True)
all_col_ann.set_index("colname", inplace=True)

In [57]:
del human_col_ann.index.name
del mouse_col_ann.index.name
del native_col_ann.index.name
del all_col_ann.index.name

In [58]:
# human_col_ann.drop("index", axis=1, inplace=True)
# mouse_col_ann.drop("index", axis=1, inplace=True)
# native_col_ann.drop("index", axis=1, inplace=True)
# all_col_ann.drop("index", axis=1, inplace=True)

In [59]:
all_col_ann.head()

,index,condition,barcode,seq
samp:HUES64_rep1__barc:10___seq:human,samp:HUES64_rep1__barc:10___seq:human,HUES64,10,human
samp:HUES64_rep1__barc:10___seq:mouse,samp:HUES64_rep1__barc:10___seq:mouse,HUES64,10,mouse
samp:HUES64_rep1__barc:11___seq:human,samp:HUES64_rep1__barc:11___seq:human,HUES64,11,human
samp:HUES64_rep1__barc:11___seq:mouse,samp:HUES64_rep1__barc:11___seq:mouse,HUES64,11,mouse
samp:HUES64_rep1__barc:12___seq:human,samp:HUES64_rep1__barc:12___seq:human,HUES64,12,human


In [60]:
all_col_ann.tail()

,index,condition,barcode,seq
samp:mESC_rep3__barc:7___seq:mouse,samp:mESC_rep3__barc:7___seq:mouse,mESC,7,mouse
samp:mESC_rep3__barc:8___seq:human,samp:mESC_rep3__barc:8___seq:human,mESC,8,human
samp:mESC_rep3__barc:8___seq:mouse,samp:mESC_rep3__barc:8___seq:mouse,mESC,8,mouse
samp:mESC_rep3__barc:9___seq:human,samp:mESC_rep3__barc:9___seq:human,mESC,9,human
samp:mESC_rep3__barc:9___seq:mouse,samp:mESC_rep3__barc:9___seq:mouse,mESC,9,mouse


## 6. make control ID files

In [61]:
ctrls = all_rna.reset_index()[["comp_id", "samp:HUES64_rep1__barc:10___seq:human"]]
ctrls["ctrl_status"] = ctrls.apply(ctrl_status, axis=1)
ctrls.drop("samp:HUES64_rep1__barc:10___seq:human", axis=1, inplace=True)
ctrls.ctrl_status.value_counts()

False    4916
True      400
Name: ctrl_status, dtype: int64

In [62]:
ctrls.head()

,comp_id,ctrl_status
0,h.2__div_lnc__m.1__div_lnc__tile1,False
1,h.3__div_lnc__m.2__antisense__tile1,False
2,h.4__antisense_upep__m.3__protein_coding__tile1,False
3,h.5__intergenic__m.4__intergenic_upep__tile1,False
4,h.6__intergenic__m.4__intergenic_upep__tile1,False


## 7. write seq comparison files

In [63]:
dna_col_ann.to_csv("%s/dna_col_ann.all_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t")
human_col_ann.to_csv("%s/HUES64_col_ann.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t")
mouse_col_ann.to_csv("%s/mESC_col_ann.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t")
native_col_ann.to_csv("%s/native_col_ann.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t")
all_col_ann.to_csv("%s/all_col_ann.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t")

ctrls.to_csv("%s/ctrl_status.all_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=False)

all_dna.to_csv("%s/dna_counts.all_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=True)
all_rna_human.to_csv("%s/HUES64_rna_counts.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=True)
all_rna_mouse.to_csv("%s/mESC_rna_counts.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=True)
all_rna_native.to_csv("%s/native_rna_counts.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=True)
all_rna.to_csv("%s/all_rna_counts.seq_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=True)

# then make files for cell line comparisons (trans effects)

## 1. run trans effects separately for human seqs & mouse seqs, so subset counts dataframe

In [64]:
human_columns = [x for x in all_rna.columns if "seq:human" in x]
mouse_columns = [x for x in all_rna.columns if "seq:mouse" in x]

In [65]:
human_trans = all_rna[human_columns]
mouse_trans = all_rna[mouse_columns]

In [66]:
print(len(human_trans))

5316


In [67]:
print(len(mouse_trans))

5316


## 2. subset annotation dataframe

In [68]:
tmp = all_col_ann
tmp.head()

,index,condition,barcode,seq
samp:HUES64_rep1__barc:10___seq:human,samp:HUES64_rep1__barc:10___seq:human,HUES64,10,human
samp:HUES64_rep1__barc:10___seq:mouse,samp:HUES64_rep1__barc:10___seq:mouse,HUES64,10,mouse
samp:HUES64_rep1__barc:11___seq:human,samp:HUES64_rep1__barc:11___seq:human,HUES64,11,human
samp:HUES64_rep1__barc:11___seq:mouse,samp:HUES64_rep1__barc:11___seq:mouse,HUES64,11,mouse
samp:HUES64_rep1__barc:12___seq:human,samp:HUES64_rep1__barc:12___seq:human,HUES64,12,human


In [69]:
human_trans_col_ann = tmp[tmp["index"].isin(human_columns)].set_index("index")
del human_trans_col_ann.index.name
human_trans_col_ann.sample(5)

,condition,barcode,seq
samp:mESC_rep1__barc:10___seq:human,mESC,10,human
samp:HUES64_rep1__barc:3___seq:human,HUES64,3,human
samp:mESC_rep3__barc:8___seq:human,mESC,8,human
samp:HUES64_rep2__barc:10___seq:human,HUES64,10,human
samp:HUES64_rep3__barc:1___seq:human,HUES64,1,human


In [70]:
mouse_trans_col_ann = tmp[tmp["index"].isin(mouse_columns)].set_index("index")
del mouse_trans_col_ann.index.name
mouse_trans_col_ann.sample(5)

,condition,barcode,seq
samp:HUES64_rep2__barc:13___seq:mouse,HUES64,13,mouse
samp:mESC_rep2__barc:5___seq:mouse,mESC,5,mouse
samp:HUES64_rep3__barc:13___seq:mouse,HUES64,13,mouse
samp:mESC_rep2__barc:6___seq:mouse,mESC,6,mouse
samp:mESC_rep2__barc:10___seq:mouse,mESC,10,mouse


In [71]:
print(len(human_columns))
print(len(human_trans_col_ann))
print(len(mouse_columns))
print(len(mouse_trans_col_ann))

78
78
78
78


## 3. write cell comparison files

In [72]:
human_trans_col_ann.to_csv("%s/human_col_ann.cell_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t")
mouse_trans_col_ann.to_csv("%s/mouse_col_ann.cell_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t")

human_trans.to_csv("%s/human_rna_counts.cell_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=True)
mouse_trans.to_csv("%s/mouse_rna_counts.cell_comp.mpranalyze.txt" % mpranalyze_dir, sep="\t", index=True)